In [ ]:
import urllib.request
import gzip
import shutil
from datetime import datetime
import os

download_path = f"../data/{datetime.today().strftime('%d-%m-%Y')}/"
os.makedirs(download_path)

imdb_dataset_domain: str = "https://datasets.imdbws.com/"
all_dataset_filenames: list[str] = ["name.basics.tsv", "title.akas.tsv", "title.basics.tsv", "title.crew.tsv", "title.episode.tsv", "title.principals.tsv", "title.ratings.tsv"]

for filename in all_dataset_filenames:
    urllib.request.urlretrieve(f"{imdb_dataset_domain}{filename}.gz", f"{download_path}{filename}.gz")
    
    with gzip.open(f"{download_path}{filename}.gz", "rb") as f_in:
        with open(f"{download_path}{filename}", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
            
    os.remove(f"{download_path}{filename}.gz")
